In [ ]:
# imports
import cv2
import matplotlib.pyplot as plt
import numpy
from sys import *
import os
from PIL import Image
from sklearn.model_selection import train_test_split
%matplotlib inline
sys.path.append ("..")

In [ ]:
def read_images(path, sz=None):
    c = 0
    X, y = [], []
    subject_path = ""
    # loop through each directory in the path and read images in each directory one by one 
    for root, dirs, files in os.walk(path):
        for subdirs in dirs:
            subject_path = os.path.join(root, subdirs)
            for filename in os.listdir(subject_path):
                try:
                    file_path = os.path.join(subject_path, filename)
                    # open image using PIL and convert it to grayscale and resize it to sz x sz pixels and append it to X and y
                    img = Image.open(file_path)
                    # single channel image so we convert it to 3 channel image by repeating the same pixel values 3 times for each channel
                    img = img.convert("L")
                    # changes pixels
                    if sz is not None:
                        # Image.ANTIALIAS smoothens the jagged edges in image by averaging the colors of the pixels at a boundary
                        img = img.resize(sz, Image.ANTIALIAS)
                    # append image to X and label to y
                    X.append(numpy.asarray(img, dtype=numpy.uint8))
                    y.append(c)
                except:
                    print("Error :", sys.exc_info()[0])
                    raise
            c = c+1
    # return the images and labels
    # print(y)
    return [X, y]

In [ ]:
# define model
model = cv2.face.LBPHFaceRecognizer_create()


In [ ]:
# select dataset and read images from it
select_dataset = int(input("Select 1 for AT&T dataset and 2 for YaleB dataset: "))
if select_dataset == 1:
    [X, y] = read_images("final_dataset/at&t")
    print("AT&T dataset loaded")
else:
    [X, y] = read_images("final_dataset/yaleB")
    print("YaleB dataset loaded")

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
# train the model on the training set
model.train(X_train, numpy.array(y_train))


In [ ]:
def display_images(l,m):
    # display the test image and the predicted person
    fig = plt.figure(figsize=(5, 5))
    fig.add_subplot(1, 2, 1)
    plt.imshow(l, cmap="gray")
    plt.axis("off")
    # display the actual person
    fig.add_subplot(1, 2, 2)
    plt.imshow(m, cmap="gray")
    plt.axis("off")
    plt.show()

total_tests = len(y_test)
correct = 0
accuracy = 0.0
# test the model on the testing set and calculate accuracy
for i in range(total_tests):
    # predict the label of the test image
    prediction, confidence = model.predict(X_test[i])
    print( " predicted =", prediction, "     "," expected =", y_test[i])
    print("    Test image        Actual person")
    display_images(X_test[i], X[prediction * 10])
    print("\n")

    # calculate accuracy
    if prediction == y_test[i]:
        correct = correct + 1

accuracy = correct
accuracy /= total_tests
accuracy *= 100
print("Accuracy: ", accuracy, "%")
